# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298540530931                   -0.85    4.9
  2   -8.300208631926       -2.78       -1.25    1.0
  3   -8.300432215986       -3.65       -1.89    3.1
  4   -8.300460774493       -4.54       -2.74    1.8
  5   -8.300463874104       -5.51       -3.01    2.8
  6   -8.300464247117       -6.43       -3.18   13.9
  7   -8.300464455373       -6.68       -3.34    6.5
  8   -8.300464551937       -7.02       -3.48    1.4
  9   -8.300464624643       -7.14       -3.71    1.8
 10   -8.300464626730       -8.68       -3.77    1.2
 11   -8.300464639880       -7.88       -4.00    1.5
 12   -8.300464643946       -8.39       -4.47    7.4


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67450006951                   -0.70    6.0
  2   -16.67864052001       -2.38       -1.14    1.9
  3   -16.67920041708       -3.25       -1.86    2.8
  4   -16.67928010214       -4.10       -2.76    2.5
  5   -16.67928603528       -5.23       -3.19    5.2
  6   -16.67928621216       -6.75       -3.48    2.4
  7   -16.67928622230       -7.99       -3.96    1.6
  8   -16.67928622423       -8.72       -4.59    2.4


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32557129900                   -0.56    6.9
  2   -33.33269695839       -2.15       -1.00    1.0
  3   -33.33409830506       -2.85       -1.73    5.1
  4   -33.33428284602       -3.73       -2.64    2.8
  5   -33.33525342746       -3.01       -2.41    4.4
  6   -33.33694316162       -2.77       -2.52    6.6
  7   -33.33694326935       -6.97       -2.53    2.5
  8   -33.33689481172   +   -4.31       -2.47    2.9
  9   -33.33682356239   +   -4.15       -2.30    2.1
 10   -33.33681888511   +   -5.33       -2.30    1.0
 11   -33.33683700295       -4.74       -2.34    1.0
 12   -33.33684474023       -5.11       -2.36    1.0
 13   -33.33686090747       -4.79       -2.40    1.0
 14   -33.33686404521       -5.50       -2.40    

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298442499873                   -0.85    4.9
  2   -8.300259018113       -2.74       -1.59    1.0
  3   -8.300419781250       -3.79       -2.49    3.2
  4   -8.300355251936   +   -4.19       -2.24    3.9
  5   -8.300463728303       -3.96       -3.33    1.1
  6   -8.300464535302       -6.09       -3.68    3.8
  7   -8.300464630084       -7.02       -4.09    1.2


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32483391075                   -0.56    6.2
  2   -33.32388266547   +   -3.02       -1.27    1.1
  3   -21.83496884571   +    1.06       -0.53    5.6
  4   -33.15142677407        1.05       -1.36    6.2
  5   -33.25033381380       -1.00       -1.38    3.1
  6   -30.15044007160   +    0.49       -0.82    4.5
  7   -33.33260126375        0.50       -2.15    4.5
  8   -33.33597630409       -2.47       -2.21    4.2
  9   -33.33332307758   +   -2.58       -2.19    2.6
 10   -33.33595648587       -2.58       -2.49    1.8
 11   -33.33686072760       -3.04       -2.96    2.6
 12   -33.33691459028       -4.27       -3.24    3.9
 13   -33.33693411091       -4.71       -3.45    2.8
 14   -33.33694234167       -5.08       -3.70    

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.